# Classificatore mediante lista

Importo le librerie necessarie per lavorare con i dati.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os.path


## Carico il dataset

In [2]:
database = pd.read_excel("Database.xlsx")
database


,Id,Compound,Class
0,1,acido cloridrico,acido
1,2,idrossido di sodio,base
2,3,etanolo,alcol
3,4,metanolo,alcol
4,5,fenolo,alcol
...,...,...,...
7,8,acido acetico,acido
8,9,mercurio,metallo
9,10,cromo,metallo
10,11,acqua,acqua


## Carico il file di input con i dati da classificare

In [3]:
input = pd.read_excel("Input.xlsx")
input


,Compound,Formula
0,acido cloridrico,HCl
1,idrossido di sodio,NaN
2,etanolo,NaN
3,metanolo,SiS
4,fenolo,OH
...,...,...
15,2-Amino-3-(3-hydroxy-5-tert-butylisoxazol-4-yl)propionic acid,C10H16N2O4
16,Clobazam,C16H13ClN2O2
17,1-(2-(4-Biphenylyloxy)ethyl)-4-phenylisonipecotic acid ethyl ester hydrobromide,C28H32BrNO3
18,sgoidngfoidsn,SiOCH3


In [4]:
import re

#removing all numbers from the string
s=input["Formula"][18]
result = re.sub(r'[0-9]+', '', s)
#split the string into a list of elements
result=re.findall('[A-Z][^A-Z]*', result)
result
for i in result:
    if "Si" in i:
        print("Yes")


Yes


In [5]:
a=0
errate_formulae=[]
for i in input["Compound"]:
    #removing all numbers from the string
    s = input["Formula"][a]
    try:
        result = re.sub(r'[0-9]+', '', s)
        #split the string into a list of elements
        result = re.findall('[A-Z][^A-Z]*', result)
    except:
        result = ""
    for i in result:
        if "Si" in i:
            errate_formulae.append(a)
            print(f"Yes {a}")
    a+=1


Yes 3
Yes 18


In [6]:
errate_formulae


[3, 18]

In [7]:
delected=[3,15,17,9,5,12] #array che generiamo nel file vecchio

In [8]:
dropped_rows = np.unique(errate_formulae + delected)
dropped_rows #tutte le righe da buttare fuori dal database

array([ 3,  5,  9, 12, 15, 17, 18])

In [10]:
input.drop(dropped_rows)


,Compound,Formula
0,acido cloridrico,HCl
1,idrossido di sodio,NaN
2,etanolo,NaN
4,fenolo,OH
6,ammoniaca,NaN
...,...,...
11,Metanolo,NaN
13,Mentolo,C10H20O
14,Aspirin,CH3COOC6H4COOH
16,Clobazam,C16H13ClN2O2


## Creole liste da popolare con i dati dopo il sorting

In [34]:
compounds=[]
class_compounds=[]

## Faccio il sorting dei dati e vedo se i composti in Input sono presenti in Database1 e come sono etichettati

In [55]:
for i in tqdm(input["Compound"]):
    compounds.append(i)
    if i in database["Compound"].to_numpy():
        class_compounds.append(
            database["Class"][database[database["Compound"] == i].index[0]])
    else:
        class_compounds.append("Unknown")


100%|██████████| 11/11 [00:00<00:00, 1832.74it/s]


In [51]:
output=pd.DataFrame(list(zip(compounds,class_compounds)),columns=["Compound","Class"])
output

,Compound,Class
0,acido cloridrico,acido
1,acido cloridrico,base
2,idrossido di sodio,alcol
3,etanolo,alcol
4,metanolo,alcol
...,...,...
6,benzene,base
7,ammoniaca,acido
8,acido acetico,metallo
9,mercurio,metallo


In [65]:
output["Class"].value_counts()

alcol        24
acido        16
base         16
metallo      16
aromatico     8
Unknown       8
Name: Class, dtype: int64

In [56]:
output.to_excel("Output.xlsx")

# Codice unico

In [98]:
def get_class(database=pd.read_excel("Database.xlsx"), input=pd.read_excel("Input.xlsx")):
    #
    # Funzione per classificare in automatico i dati in base ai parametri contenuti nel database
    #
    # database: database su cui si basa la classificazione
    # input: dati di output
    # output: dati classificati
    #

    #liste vuote usate nella classificazione
    compounds = []  # lista di tutti i composti
    class_compounds = []  # lista di tutte le classi dei composti

    for i in tqdm(input["Compound"]):  # leggo i composti dall'input
        # aggiungo il nome del i composto nella lista dei composti
        compounds.append(i)
        # controllo se il composto i è presente nel database, se si aggiungo la classe conosciuta
        if i in database["Compound"].to_numpy():
            class_compounds.append(
                database["Class"][database[database["Compound"] == i].index[0]])
        else:  # se no, classifica il composto come sconosciuto
            class_compounds.append("Unknown")

    output = pd.DataFrame(list(zip(compounds, class_compounds)),
                          columns=["Compound", "Class"])  # creo il dataframe con i dati classificati

    print("Report of your input:")
    # print della descrizione dei composti classificati
    print(output["Class"].value_counts())
    print("\n")

    if os.path.isfile('Output.xlsx'):  # controllo se esiste già un file di output
        print("File \"Output.xlsx\" alredy exists in this directory.\nIf you want to obtain a new one, delete it.")
    else:
        print("File \"Output.xlsx\" will be saved in this directory.")
        # converto il dataframe in un file excel
        output.to_excel("Output.xlsx")

    print("\nSee you next time ;)")



In [99]:
get_class()

100%|██████████| 11/11 [00:00<00:00, 2202.05it/s]

Report of your input:
alcol        3
acido        2
base         2
metallo      2
aromatico    1
Unknown      1
Name: Class, dtype: int64


File "Output.xlsx" alredy exists in this directory.
If you want to obtain a new one, delete it.

See you next time ;)
